In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dataset

In [14]:
import os
import json
import pandas as pd

In [83]:
PATH = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft/data/"
STORAGE_PATH ="/content/drive/MyDrive/fine_tuning_llm_with_dop_peft/data/refactored_dataset/"
TRAIN_FILE_PATH = os.path.join(PATH, "train.json")
TEST_FILE_PATH = os.path.join(PATH, "test.json")
DEV_FILE_PATH = os.path.join(PATH, "dev.json")

In [80]:
question = []
answer = []
for file_path in [TRAIN_FILE_PATH, TEST_FILE_PATH, DEV_FILE_PATH]:
  with open(file_path, "r") as fp:
    data = json.load(fp)

  for i in range(len(data)):
    if data[i]["qa"]["answer"]  != '':
      question.append(data[i]["qa"]["question"])
      answer.append(data[i]["qa"]["answer"])

question_answer = dict(zip(question, answer))
df = pd.DataFrame.from_dict(question_answer.items())
df_prompt = df.rename(columns = {0:"prompt", 1:"text"})

# let's create rejected text
df_prompt["rejected_text"] = df_prompt["text"].shift(-1)
df_prompt.loc[df_prompt["rejected_text"].shape[0]-1,"rejected_text"] = df_prompt.iloc[df_prompt["rejected_text"].shape[0]-3]["rejected_text"]

In [84]:
os.makedirs(STORAGE_PATH, exist_ok=True)
df_prompt.to_csv(os.path.join(STORAGE_PATH, "train.csv"), index=False)